In [ ]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Acessa a base de sócios e administradores da Receita Federal
# Separa empresas ativas do AM e apenas de Parintins
#

In [1]:
import pandas as pd

In [ ]:
# Arquivo de QSA da Receita coletado em 
# https://github.com/georgevbsantiago/qsacnpj

In [2]:
# Empresas ativas do AM

In [ ]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/maio_2020/cnpj_dados_cadastrais_pj.csv',\
                             chunksize=100000,\
                             sep='#',\
                             dtype= str,\
                            decimal=',',
                             usecols=['cnpj',
                                      'razao_social',
                                      'nome_fantasia',
                                      'situacao_cadastral',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio', 
                                     'capital_social_empresa'])

In [ ]:
now = datetime.now()
print("A extração começou às: ", now)

# Estabelece contagem de vezes
conta = 0

# Iteração em cada chunk
for df in TextFileReader:
        
    identificador = df['uf'].isin(['AM']) & (df['situacao_cadastral'] == '02')
    nome = df[identificador]
    
    # Testa se encontrou algo
    if nome.empty is False:
        # Se for a primeira vez cria o dataframe df_procura1
        if conta == 0:
            df_procura1 = nome
        # Se não for a primeira vez faz append no dataframe df_procura1
        else:
            df_procura1 = df_procura1.append(nome)
    
        conta = conta + 1
    
now = datetime.now()
print("A extração terminou às: ", now)

In [ ]:
df_procura1.shape

In [ ]:
TextFileReader = pd.read_csv('/home/abraji/Documentos/Code/qsa_receita/maio_2020/cnpj_dados_socios_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'identificador_socio',
                                      'nome_socio',
                                      'cnpj_cpf_socio',
                                      'cod_qualificacao_socio',
                                      'data_entrada_sociedade',
                                      'cpf_representante_legal',
                                      'nome_representante'])

In [ ]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    socios_encontrados = pd.merge(df_procura1, \
                                 df, \
                                 left_on='cnpj', \
                                 right_on='cnpj')
    
    if socios_encontrados.empty is False:
        if conta == 0:
            df_procura_final = socios_encontrados
        else:
            df_procura_final = df_procura_final.append(socios_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

In [ ]:
df_procura_final.info()

In [ ]:
# Apenas Parintins

In [ ]:
parintins = df_procura_final[(df_procura_final['municipio'] == 'PARINTINS')]

In [ ]:
parintins.to_excel('resultados/empresas_ativas_parintins_am.xlsx',sheet_name='Sheet1', index=False)